In [1]:
import numpy as np
import random
import platform
import sys
import os
import math
import time
import matplotlib
from matplotlib import pyplot as plt
import vtk

In [ ]:
raw_data_file = 'data/yA31_v02_300x300x300_99.vtk'
sampled_file = 'data/asteroid_sampled_hist_grad0.02.vtp'
var_name = 'vo2'
var_name1 = 'vo2'
query_th1 = 0.6
query_th2 = 1

In [2]:
raw_data_file = '/home/soumya/Test_DataSet/multivar_sampling_test_data/Isabel_pressure_velocity_qvapor.vti'
sampled_file = 'isabel_sampled.vtp'
var_name = 'Pressure'
var_name1 = 'Pressure_smooth'
query_th1 = -50
query_th2 = -5000

In [3]:
## load data
reader = vtk.vtkXMLImageDataReader()
reader.SetFileName(raw_data_file)
reader.Update()
raw_data = reader.GetOutput()

reader1 = vtk.vtkXMLPolyDataReader()
reader1.SetFileName(sampled_file)
reader1.Update()
sampled_pts = reader1.GetOutput()

extent = raw_data.GetExtent()
spacing = raw_data.GetSpacing()
origin = raw_data.GetOrigin()
dims = raw_data.GetDimensions()

In [4]:
def compute_3d_to_1d_map(x,y,z,dimx,dimy,dimz):
    index = x + dimx*(y+dimy*z)
    return index

In [5]:
## create new image data
new_data = vtk.vtkImageData()
new_data.SetDimensions(dims)
new_data.SetSpacing(spacing)
new_data.SetOrigin(origin)

## create point arrays
query_pts = vtk.vtkPoints()
query_pts_physical = vtk.vtkPoints()
query_arr = vtk.vtkDoubleArray()
query_arr.SetName(var_name)

query_arr_new = vtk.vtkDoubleArray()
query_arr_new.SetName(var_name)
query_arr_new.SetNumberOfTuples(dims[0]*dims[1]*dims[2])

query_arr_new1 = vtk.vtkDoubleArray()
query_arr_new1.SetName(var_name1)
query_arr_new1.SetNumberOfTuples(dims[0]*dims[1]*dims[2])

for i in range(sampled_pts.GetNumberOfPoints()):
    vo2 = sampled_pts.GetPointData().GetArray(var_name).GetTuple1(i)
    #if vo2 >= query_th1 and vo2 <= query_th2:
    if vo2 >= query_th2 and vo2 <= query_th1:    
        pts = sampled_pts.GetPoint(i)        
        ii = int((pts[0] - origin[0])/spacing[0] + 0.5)
        jj = int((pts[1] - origin[1])/spacing[1] + 0.5)
        kk = int((pts[2] - origin[2])/spacing[2] + 0.5)
        query_pts.InsertNextPoint([ii,jj,kk])
        query_arr.InsertNextTuple1(vo2)   
        query_pts_physical.InsertNextPoint(pts)
        
        index = compute_3d_to_1d_map(ii,jj,kk,dims[0],dims[1],dims[2])
        query_arr_new.SetTuple1(index,vo2)
        
        query_arr_new1.SetTuple1(index,vo2)
        
        ## add values to the new grid also
        #new_data.SetScalarComponentFromDouble(ii,jj,kk,0,vo2)

#new_data.GetPointData().AddArray(query_arr_new)         
print (query_pts.GetNumberOfPoints())  

pdata = vtk.vtkPolyData()
pdata.SetPoints(query_pts_physical)
pdata.GetPointData().AddArray(query_arr)

writer1 = vtk.vtkXMLPolyDataWriter()
writer1.SetInputData(pdata)
writer1.SetFileName('output/recon_query.vtp')
writer1.Write()

print query_arr_new1.GetRange()

37905
(-4931.54248046875, 0.0)


In [6]:
count_arr = vtk.vtkIntArray()
count_arr.SetName(var_name1)
count_arr.SetNumberOfTuples(dims[0]*dims[1]*dims[2])

window = 2

for p  in range(query_pts.GetNumberOfPoints()):

    pts = query_pts.GetPoint(p)
    vo2 = query_arr_new.GetTuple1(p)
    
    i=int(pts[0])
    j=int(pts[1])
    k=int(pts[2])
    
    cc=0
    
    for kk in range(k-window,k+window):
        for jj in range(j-window,j+window):
            for ii in range(i-window,i+window):
                cc = cc+1
                ## check if in bound
                if ii >= 0 and ii < dims[0] and jj >= 0 and jj < dims[1] and kk >= 0 and kk <dims[2]:
                    
                    ## ignore the central point
                    if kk == k and ii== i and jj == j: 
                        continue
                    else:
                        index1 = compute_3d_to_1d_map(ii,jj,kk,dims[0],dims[1],dims[2])

                        if query_arr_new.GetTuple1(index1) == 0:
                            #print 'here'
                            v2 = query_arr_new1.GetTuple1(index1)
                            v2 = v2 + vo2
                            query_arr_new1.SetTuple1(index1,v2)
                            
                            count_arr.SetTuple1(index1,count_arr.GetTuple1(index1)+1)
    #print cc               
    
   

In [7]:
print query_arr_new1.GetRange()

for i in range(query_arr_new1.GetNumberOfTuples()):
    v = query_arr_new1.GetTuple1(i)
    c = count_arr.GetTuple1(i)
    if c  > 0:
        query_arr_new1.SetTuple1(i,v/64)

new_data.GetPointData().AddArray(query_arr_new1)    


print query_arr_new1.GetRange()

(-4931.54248046875, 0.0)
(-4931.54248046875, 0.0)


In [8]:
writer = vtk.vtkXMLImageDataWriter()
writer.SetInputData(new_data)
writer.SetFileName('output/recon_query.vti')
writer.Write()   

1